```{contents}
```
## Chain-of-Thought (CoT) 


**Chain-of-Thought (CoT)** is a prompting technique where the model is encouraged to **reason step by step** before producing a final answer.

> The goal is to improve **reasoning quality**, not to expose reasoning verbatim.

From a **LangChain perspective**, CoT is **controlled reasoning orchestration**, not free-form explanation dumping.

---

### Why Chain-of-Thought Exists

LLMs struggle with:

* Multi-step logic
* Math and reasoning
* Conditional decisions

CoT improves:

* Accuracy
* Logical consistency
* Complex task handling

---

### Critical Safety Rule (Very Important)

> **Do NOT expose raw chain-of-thought to users in production.**

LangChain follows **reasoning safety best practices**:

* Reason internally
* Return **concise answers**
* Optionally return **summaries of reasoning**

---

### Chain-of-Thought vs Explanation

| Concept          | Purpose                   |
| ---------------- | ------------------------- |
| Chain-of-Thought | Internal reasoning        |
| Explanation      | User-facing justification |

LangChain encourages **hidden CoT + summarized output**.

---

### Types of Chain-of-Thought

### Zero-shot CoT

Encourages reasoning without examples.

```text
Let's think step by step.
```

---

### Few-shot CoT

Provides examples **with reasoning steps**.

---

### Structured CoT

Reasoning is guided but **not exposed**.

---

### Zero-shot CoT Demonstration (Simple)

#### Prompt



In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a logical reasoning assistant."),
    ("human", "Solve the problem step by step, then give the final answer only.\n\nQuestion: {input}")
])

chain = prompt | llm

response = chain.invoke({"input": "If a ticket takes 2 hours and there are 3 tickets, how long?"})
print(response.content)


To find the total time for 3 tickets when each ticket takes 2 hours, we can follow these steps:

1. Determine the time for one ticket: 2 hours.
2. Multiply the time for one ticket by the number of tickets: 
   \( 2 \text{ hours/ticket} \times 3 \text{ tickets} = 6 \text{ hours} \).

Final answer: 6 hours.




### Model Behavior

* Performs internal reasoning
* Outputs final answer only

---

### Few-shot Chain-of-Thought Demonstration

#### Prompt with Examples



In [2]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You solve problems logically."),
    ("human", "If one ticket takes 1 hour and there are 2 tickets, how long?"),
    ("assistant", "First determine time per ticket. Then multiply by count. Final answer: 2 hours."),
    ("human", "{input}")
])



This teaches:

* Reasoning pattern
* Answer structure

---

### Chain-of-Thought with Output Parsers (Best Practice)

#### Reason Internally, Return Structured Answer



In [3]:

from pydantic import BaseModel

class Answer(BaseModel):
    result: str

structured_llm = llm.with_structured_output(Answer)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Reason internally. Return only the final result."),
    ("human", "{input}")
])

chain = prompt | structured_llm

result = chain.invoke({"input": "If SLA is 4h and delay is 1h, total?"})
print(result)


result='5h'




No reasoning leakage.

---

### Chain-of-Thought in Agents

### Internal Agent Reasoning

Agents **always use CoT internally**:

```
Thought → Tool → Observation → Thought → Answer
```

But LangChain:

* Stores it in `intermediate_steps`
* Does NOT expose it by default

---

### Chain-of-Thought in LangGraph

LangGraph nodes:

* Perform reasoning internally
* Pass state forward
* Return only safe outputs

This is the **production-grade CoT approach**.

---

### What NOT to Do (Anti-patterns)

- Exposing raw CoT

```text
Here is my reasoning step by step...
```

- Logging CoT to users

Security + compliance risk.

- Relying on CoT for determinism

Use **structured outputs** instead.

---

### Correct Production Pattern

```
User Input
  ↓
Hidden CoT
  ↓
Validation / Parsing
  ↓
Concise Answer
```

---

### When to Use Chain-of-Thought

| Scenario            | Use CoT           |
| ------------------- | ----------------- |
| Math / logic        | Yes               |
| Planning            | Yes               |
| Agents              | Always (internal) |
| Simple Q&A          | No                |
| RAG factual answers | Minimal           |

---

### Chain-of-Thought vs Related Techniques

### CoT vs ReAct

| Aspect           | CoT      | ReAct       |
| ---------------- | -------- | ----------- |
| Reasoning        | Internal | Interleaved |
| Tools            | ❌        | ✅           |
| LangChain agents | Internal | Preferred   |

---

### CoT vs Tree of Thoughts

| Aspect     | CoT    | ToT      |
| ---------- | ------ | -------- |
| Paths      | Single | Multiple |
| Cost       | Lower  | Higher   |
| Complexity | Medium | High     |

---

### Best Practices (LangChain)

* Let the model reason internally
* Use structured outputs
* Use agents for tool reasoning
* Never expose raw reasoning
* Log reasoning only for debugging (restricted)

---

### Interview-Ready Summary

> “Chain-of-thought is an internal reasoning technique that improves LLM accuracy on multi-step problems. In LangChain, CoT is used implicitly in agents and reasoning chains, while only concise, safe outputs are returned to users.”

---

### Rule of Thumb

* **Reason internally**
* **Respond concisely**
* **Structure outputs**
* **Never leak chain-of-thought**

